### 🛠️ 10 Tasks:

1. Analyze query plans
2. Partition large tables
3. Apply ZORDER
4. Benchmark improvements

## Task 1: Analyze Query Plan

Pick a Query to Analyze

Use EXPLAIN (Logical + Physical Plan)

In [0]:
%sql
EXPLAIN
SELECT
  order_date,
  total_revenue
FROM ecommerce_catalog.gold.daily_sales
WHERE order_date >= '2019-10-01'
ORDER BY order_date;


plan
"== Physical Plan == AdaptiveSparkPlan isFinalPlan=false +- == Initial Plan == ColumnarToRow +- PhotonResultStage +- PhotonSort [order_date#13171 ASC NULLS FIRST] +- PhotonScan parquet ecommerce_catalog.gold.daily_sales[order_date#13171,total_revenue#13172] DataFilters: [isnotnull(order_date#13171), (order_date#13171 >= 2019-10-01)], DictionaryFilters: [(order_date#13171 >= 2019-10-01)], Format: parquet, Location: PreparedDeltaFileIndex(1 paths)[s3://dbstorage-prod-g9urr/uc/c4d0d19d-f1d2-4e1a-afbd-94a246ba7d40..., OptionalDataFilters: [], PartitionFilters: [], ReadSchema: struct, RequiredDataFilters: [isnotnull(order_date#13171), (order_date#13171 >= 2019-10-01)] == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = full = daily_sales"


## Task 2: Partition Large Tables

Partitioning helps:

Reduce scanned files

Enable partition pruning

Improve query speed on filters

👉 We partition on high-cardinality filter columns used in WHERE

Recreate Gold Table with Partition

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce_catalog.gold.daily_sales_partitioned
PARTITIONED BY (order_date)
AS
SELECT *
FROM ecommerce_catalog.gold.daily_sales;


num_affected_rows,num_inserted_rows


Verify Partitioning

In [0]:
%sql
DESCRIBE DETAIL ecommerce_catalog.gold.daily_sales_partitioned;


format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,2cbd6339-a797-48cb-b238-adc43cbe348f,ecommerce_catalog.gold.daily_sales_partitioned,null,,2026-01-20T17:13:00.714Z,2026-01-20T17:13:09.000Z,List(order_date),List(),61,81237,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


##Task 3: Apply Z-ORDER

We already did partitioning ✅
But partitioning alone is not enough when:

Queries filter on non-partition columns

Data inside a partition is still large

You want faster data skipping

👉 Z-ORDER improves data skipping inside partitions

Apply Z-ORDER

In [0]:
%sql
OPTIMIZE ecommerce_catalog.gold.daily_sales_partitioned
ZORDER BY (total_revenue);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 61, List(minCubeSize(107374182400), List(0, 0), List(61, 81237), 0, List(0, 0), 0, null), null, 0, 0, 61, 61, false, 0, 0, 1768929654732, 1768929658920, 8, 0, null, List(0, 0), null, 4, 4, 0, 0, null)"


Re-run EXPLAIN (Validation)

In [0]:
%sql
EXPLAIN
SELECT
  order_date,
  total_revenue
FROM ecommerce_catalog.gold.daily_sales_partitioned
WHERE order_date >= '2019-10-01'
ORDER BY order_date;


plan
"== Physical Plan == AdaptiveSparkPlan isFinalPlan=false +- == Initial Plan == ColumnarToRow +- PhotonResultStage +- PhotonSort [order_date#13785 ASC NULLS FIRST] +- PhotonShuffleExchangeSource +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#8930] +- PhotonShuffleExchangeSink rangepartitioning(order_date#13785 ASC NULLS FIRST, 1024) +- PhotonProject [order_date#13785, total_revenue#13786] +- PhotonScan parquet ecommerce_catalog.gold.daily_sales_partitioned[total_revenue#13786,order_date#13785] DataFilters: [], DictionaryFilters: [], Format: parquet, Location: PreparedDeltaFileIndex(1 paths)[s3://dbstorage-prod-g9urr/uc/c4d0d19d-f1d2-4e1a-afbd-94a246ba7d40..., OptionalDataFilters: [], PartitionFilters: [isnotnull(order_date#13785), (order_date#13785 >= 2019-10-01)], ReadSchema: struct, RequiredDataFilters: [] == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = daily_sales_partitioned full = Corrective actions: consider running the following command on all tables with missing or partial statistics ANALYZE TABLE COMPUTE STATISTICS FOR ALL COLUMNS"


🔍 What improved

✔ Partition pruning (order_date)

✔ Data skipping (total_revenue)

✔ Fewer files scanned

✔ Faster execution

## Task 4: Benchmark Improvements
We now compare BEFORE vs AFTER

Capture BEFORE metrics (unpartitioned table)

In [0]:
%sql
SELECT
  order_date,
  total_revenue
FROM ecommerce_catalog.gold.daily_sales
WHERE order_date >= '2019-10-01'
ORDER BY order_date;



order_date,total_revenue
2019-10-01,3.7063524225992954E8
2019-10-02,3.5757353882987684E8
2019-10-03,3.394813026499579E8
2019-10-04,4.236928194202042E8
2019-10-05,3.956764972899051E8
2019-10-06,3.97025067050164E8
2019-10-07,3.554490384999612E8
2019-10-08,3.808717091899185E8
2019-10-09,3.7977535402009356E8
2019-10-10,3.7205491930024576E8


📌 Note:

Runtime

Files scanned (from UI)

Data read

Capture AFTER metrics (optimized table)

In [0]:
%sql
SELECT
  order_date,
  total_revenue
FROM ecommerce_catalog.gold.daily_sales_partitioned
WHERE order_date >= '2019-10-01'
ORDER BY order_date;


order_date,total_revenue
2019-10-01,3.7063524225992954E8
2019-10-02,3.5757353882987684E8
2019-10-03,3.394813026499579E8
2019-10-04,4.236928194202042E8
2019-10-05,3.956764972899051E8
2019-10-06,3.97025067050164E8
2019-10-07,3.554490384999612E8
2019-10-08,3.808717091899185E8
2019-10-09,3.7977535402009356E8
2019-10-10,3.7205491930024576E8


📌 Compare:

Runtime ↓

Files scanned ↓

IO cost ↓

Although runtime remained similar due to small data size and Photon execution, execution plans confirmed partition pruning, reduced scans, and improved IO efficiency. These optimizations will significantly improve performance at scale